# Generate Region Availability File

This utility generates random availability information for muultiple periods to be displayed for regions on maps. Availability can be anything, such as energy.

In [1]:
import pandas as pd
import geopandas as gpd
import random
import json

/Users/peterjanes/.local/share/virtualenvs/python38-O-C_Z_QB/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
# reading regions (only used to get region identifiers and names)
# Swiss Kantons for this example, but can be any region list (for later display as shapes on map)
shapeFilename = "SHAPEFILE_LV95_LN02/swissBOUNDARIES3D_1_3_TLM_KANTONSGEBIET.shp"

# generating availability file
# schema filename
schemaFilename = "AvailabilityMapSchema.json"
# availability filename
availabilityFilename = "AvailabilityMapProvider3.json"

In [3]:
# availability information provider
providerId = 789
providerName = "Provider 3"

In [4]:
# availability period parameters
noOfPeriods = 5
startDate = "2022-12-06"
intervalHours = 4

## Load Regions

Using Kantons in this example, but could be any region

In [5]:
shape_regions = gpd.read_file(shapeFilename)

In [6]:
shape_regions.head()

,UUID,DATUM_AEND,DATUM_ERST,ERSTELL_J,ERSTELL_M,REVISION_J,REVISION_M,GRUND_AEND,HERKUNFT,HERKUNFT_J,...,OBJEKTART,REVISION_Q,ICC,KANTONSNUM,SEE_FLAECH,KANTONSFLA,KT_TEIL,NAME,EINWOHNERZ,geometry
0,{0B2364ED-49E0-4D53-A33C-C684DD530B57},2018-11-22,2012-10-26,2012,10,2022,5,Verbessert,AV,2019,...,Kanton,2021_Aufbau,CH,18,NaN,710530.0,0,Graubünden,200096.0,"POLYGON Z ((2735215.665 1194955.161 2870.714, ..."
1,{DDD56CEF-0E61-4EED-85ED-F67A459C93ED},2021-11-02,2012-10-26,2012,10,2022,5,Verbessert,AV,2021,...,Kanton,2021_Aufbau,CH,2,11897.0,595850.0,1,Bern,1043081.0,"POLYGON Z ((2595241.510 1169313.282 2090.846, ..."
2,{54B25E50-30A7-4995-ADE3-5FFF6E13A995},2018-11-22,2012-10-26,2012,10,2022,5,Verbessert,AV,2019,...,Kanton,2021_Aufbau,CH,23,1060.0,522463.0,0,Valais,348503.0,"POLYGON Z ((2601807.628 1136116.777 2948.413, ..."
3,{921DFEF2-6D91-4CB8-9CFC-2A831C412020},2019-11-06,2012-10-26,2012,10,2022,5,Verbessert,swisstopo,2019,...,Kanton,2021_Aufbau,CH,22,39097.0,321202.0,1,Vaud,814762.0,"POLYGON Z ((2555092.813 1138713.419 372.133, 2..."
4,{95F10F52-8B2F-4D6A-AF7E-D4F915E42F89},2020-11-19,2012-10-26,2012,10,2022,5,Verbessert,AV,2020,...,Kanton,2021_Aufbau,CH,21,7147.0,281215.0,0,Ticino,350986.0,"POLYGON Z ((2727358.511 1119219.094 1351.527, ..."


In [7]:
# shapes actually not required - just creating list of regions with unique identifiers and names
r_list = pd.DataFrame(shape_regions,columns=['NAME','KANTONSNUM']).drop_duplicates(subset = "NAME")
r_list.head()

,NAME,KANTONSNUM
0,Graubünden,18
1,Bern,2
2,Valais,23
3,Vaud,22
4,Ticino,21


In [8]:
# generating period sequence with ramdom values
def random_period(p):
    p_list = [False] * p
    for pl in range(len(p_list)):
        p_list[pl] = bool(random.getrandbits(1))
    return p_list

In [9]:
# gererate random period sequences for all regions
r_settings = []
for i, rg in r_list.iterrows():
    r_settings.append([[rg['KANTONSNUM'], rg['NAME']], random_period(noOfPeriods)])
r_settings

[[[18, 'Graubünden'], [True, True, True, False, True]],
 [[2, 'Bern'], [False, False, False, True, False]],
 [[23, 'Valais'], [True, True, False, False, False]],
 [[22, 'Vaud'], [False, False, False, False, True]],
 [[21, 'Ticino'], [True, False, True, False, True]],
 [[17, 'St. Gallen'], [True, True, True, True, False]],
 [[1, 'Zürich'], [False, False, True, False, False]],
 [[10, 'Fribourg'], [False, False, True, False, True]],
 [[3, 'Luzern'], [False, True, False, False, False]],
 [[19, 'Aargau'], [False, True, True, False, True]],
 [[4, 'Uri'], [False, False, False, False, True]],
 [[20, 'Thurgau'], [False, False, False, False, False]],
 [[5, 'Schwyz'], [True, True, True, True, True]],
 [[26, 'Jura'], [True, True, False, True, False]],
 [[24, 'Neuchâtel'], [False, True, True, True, False]],
 [[11, 'Solothurn'], [False, True, False, False, True]],
 [[8, 'Glarus'], [True, True, False, False, False]],
 [[13, 'Basel-Landschaft'], [True, True, False, False, True]],
 [[6, 'Obwalden'], [T

## Creating structure, writing to json file

In [10]:
regionAvailabilities = {
    "$schema": schemaFilename,
    "provider": {
        "providerId": providerId,
        "providerName": providerName
    },
    "regionPeriods": {
        "noOfPeriods": noOfPeriods,
        "startDate": startDate,
        "intervalHours": intervalHours,
        "regionPeriodValues": []
    }
}

regionPeriodValues = []
for r in range(len(r_settings)):
    regVal = { "region": { "regionId": r_settings[r][0][0], "regionName": r_settings[r][0][1] }, "periodValues": r_settings[r][1] }
    regionAvailabilities['regionPeriods']['regionPeriodValues'].append(regVal)
regionAvailabilities

{'$schema': 'AvailabilityMapSchema.json',
 'provider': {'providerId': 789, 'providerName': 'Provider 3'},
 'regionPeriods': {'noOfPeriods': 5,
  'startDate': '2022-12-06',
  'intervalHours': 4,
  'regionPeriodValues': [{'region': {'regionId': 18,
     'regionName': 'Graubünden'},
    'periodValues': [True, True, True, False, True]},
   {'region': {'regionId': 2, 'regionName': 'Bern'},
    'periodValues': [False, False, False, True, False]},
   {'region': {'regionId': 23, 'regionName': 'Valais'},
    'periodValues': [True, True, False, False, False]},
   {'region': {'regionId': 22, 'regionName': 'Vaud'},
    'periodValues': [False, False, False, False, True]},
   {'region': {'regionId': 21, 'regionName': 'Ticino'},
    'periodValues': [True, False, True, False, True]},
   {'region': {'regionId': 17, 'regionName': 'St. Gallen'},
    'periodValues': [True, True, True, True, False]},
   {'region': {'regionId': 1, 'regionName': 'Zürich'},
    'periodValues': [False, False, True, False, Fals

In [11]:
# Serializing json
json_object = json.dumps(regionAvailabilities, indent=2, ensure_ascii=False)

# Writing to file
with open(availabilityFilename, "w") as outfile:
    outfile.write(json_object)